# **INTRODUCTIONS**

---

The third vacchination shots are represented on the **[vaccination page](https://coronadashboard.rijksoverheid.nl/landelijk/vaccinaties)**.   

The source data (i.e. **[RIVM](https://www.rivm.nl/en)**) is modelled on daily registrations of the third vaccination shots at **GGD**.

The source code is separated into multiple sections:

1. **[Tables](#tables)**
2. **[Views](#views)**
3. **[Stored Procedures](#stored-procedures)**
4. **[Drop Artifacts](#drop-artifacts)**
5. **[Datatino Configurations](#datatino-configurations)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/conversions.ipynb",
        "src/utils/schemas.ipynb",
        "src/utils/protos.ipynb",
        "src/statics/reclassified_municipality_mapping.ipynb"
    ]
}
```

# **TABLES**

---


### STAGING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

IF NOT EXISTS(SELECT * FROM SYS.SEQUENCES WHERE OBJECT_ID = OBJECT_ID(N'[DBO].[SEQ_VWSSTAGE_NICE_HOSPITAL]') AND TYPE = 'SO')
CREATE SEQUENCE [SEQ_VWSSTAGE_NICE_HOSPITAL]
	START WITH 1
	INCREMENT BY 1;
GO

IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[NICE_HOSPITAL]'))
CREATE TABLE [VWSSTAGE].[NICE_HOSPITAL](
	[ID] INT PRIMARY KEY NOT NULL DEFAULT (NEXT VALUE FOR [DBO].[SEQ_VWSSTAGE_NICE_HOSPITAL]),
	[VERSION] VARCHAR(100) NULL,
	DATE_OF_REPORT VARCHAR(100) NULL,
	DATE_OF_STATISTICS VARCHAR(100) NULL,
	MUNICIPALITY_CODE VARCHAR(100) NULL,
	MUNICIPALITY_NAME VARCHAR(100) NULL,
	SECURITY_REGION_CODE VARCHAR(100) NULL,
	SECURITY_REGION_NAME VARCHAR(100) NULL,
	HOSPITAL_ADMISSION_NOTIFICATION VARCHAR(100) NULL,
	HOSPITAL_ADMISSION VARCHAR(100) NULL,
	DATE_LAST_INSERTED DATETIME DEFAULT GETDATE()
);
GO

### INTERMEDIATES

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

IF NOT EXISTS(SELECT * FROM SYS.SEQUENCES WHERE OBJECT_ID = OBJECT_ID(N'[DBO].[SEQ_VWSINTER_NICE_HOSPITAL]') AND TYPE = 'SO')
CREATE SEQUENCE SEQ_VWSINTER_NICE_HOSPITAL
	START WITH 1
	INCREMENT BY 1;
GO

IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSINTER].[NICE_HOSPITAL]'))
CREATE TABLE [VWSINTER].[NICE_HOSPITAL](
	[ID] INT PRIMARY KEY NOT NULL DEFAULT (NEXT VALUE FOR [DBO].[SEQ_VWSINTER_NICE_HOSPITAL]),
	DATE_OF_REPORT DATETIME NULL,
	DATE_OF_STATISTICS DATETIME NULL,
	MUNICIPALITY_CODE VARCHAR(100) NULL,
	MUNICIPALITY_NAME VARCHAR(100) NULL,
	SECURITY_REGION_CODE VARCHAR(100) NULL,
	SECURITY_REGION_NAME VARCHAR(100) NULL,
	REPORTED INT NULL,
	HOSPITALIZED INT NULL,
	DATE_LAST_INSERTED DATETIME DEFAULT GETDATE()
);
GO

# **STORE PROCEDURES**
---

### STAGING &rarr; INTERMEDIATE MAPPING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE OR ALTER PROCEDURE [DBO].[SP_NICE_HOSPITAL_INTER]
AS
BEGIN

-- MAIN SELECT AND INSERT INTO STATEMENT. FILTERED VALUES ON MAX DATELASTINSERTED.
-- MOVE INFECTIOUS DATA FROM STAGING TO INTERMEDIATE TABLE.
INSERT INTO [VWSINTER].[NICE_HOSPITAL](
	[DATE_OF_REPORT]
    ,[DATE_OF_STATISTICS]
    ,[MUNICIPALITY_CODE]
    ,[MUNICIPALITY_NAME]
    ,[SECURITY_REGION_CODE]
    ,[SECURITY_REGION_NAME]
    ,[REPORTED]
    ,[HOSPITALIZED]
)
    SELECT
        CAST(A.[DATE_OF_REPORT] AS [DATETIME])
        ,CAST(A.[DATE_OF_STATISTICS] AS [DATETIME])
        ,ISNULL(B.[GM_CODE_NEW], A.[MUNICIPALITY_CODE])
        ,ISNULL(B.[GM_NAME_NEW], A.[MUNICIPALITY_NAME])
        ,ISNULL(B.[VR_CODE], A.[SECURITY_REGION_CODE])
        ,ISNULL(B.[VR_NAME], A.[SECURITY_REGION_NAME])
        ,SUM(CAST([HOSPITAL_ADMISSION_NOTIFICATION] AS [INT])) AS [REPORTED]
        ,SUM(CAST([HOSPITAL_ADMISSION] AS [INT])) AS [HOSPITALIZED]
    FROM [VWSSTAGE].[NICE_HOSPITAL] A
    LEFT JOIN (
        SELECT 
            * 
        FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] 
        WHERE [ACTIVE]=1
            AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING])  
    ) B ON A.[MUNICIPALITY_CODE] = B.[GM_CODE_ORIGINAL]
    WHERE A.[DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[NICE_HOSPITAL])
    GROUP BY ISNULL(B.[GM_CODE_NEW], A.[MUNICIPALITY_CODE])
            ,ISNULL(B.[GM_NAME_NEW], A.[MUNICIPALITY_NAME])
            ,ISNULL(B.[VR_CODE], A.[SECURITY_REGION_CODE])
            ,ISNULL(B.[VR_NAME], A.[SECURITY_REGION_NAME])
            ,A.[DATE_OF_REPORT]
            ,A.[DATE_OF_STATISTICS]
END;
GO